# Transformers

The main feature that the Transformer makes use of is an attention mechanism, which is a kind of layer operation that we build into the Transformer model that allows the model to explicitly select certain features to focus on when making its predictions. This mechanism also gives us an extra degree of interpretability, in the sense that we can look inside the network to examine which parts of the input it's attending to.

The Transformer was proposed as a model for machine transition, and also natural language parsing, although the authors did suggest that it could also be applicable for other tasks. And the Transformer has since been further extended from these original applications, to problems such as image classification, image captioning, and generative models of natural language and images, even conditioned on a caption input.

Let's talk about the natural language processing setting. Even with LSTM, there is a long signal path that information has to travel between two events that are many time steps apart. The path grows linearly in the number of time steps separating the two events. So even though the LSTM goes a long way to mitigate this problem with its internal memory cell, in practice it still struggles to successfully model very long range dependencies that you might find for example in a text document, where the meaning in one part of the document might depend on information that came several sentences or even paragraphs earlier.

Moreover, with recurrent neural networks it is not possible to independently distribute the computation on different parts of the sequence to diffent devices, because of the inherently sequential nature of the model, and the fact that each hidden state depends on all the previous hidden states in the sequence. With convolutional neural networks, model parallelisation, sending parts of the model computation to different devices that are available, is more feasible. However, in a CNN, the signal path that connects information between two events that are many time steps apart, can be very long. So CNNs will also struggle to model long-range dependencies when applied to sequential data.

Lastly, both CNNs and RNNs lack interpretability. For high-dimensional data inputs that we need to reduce the dimensionality, we use pooling operations (CNNs) or use cell states (LSTMs) and hope they retain the relevant information for as long as it needs to, and discards whatever is irrelevant. With the attention mechanism of the Transformer, it becomes easy to examine what parts of the data input the model is connecting together, and where it is looking in order to make its predictions at each time step.

### Overall Architecture

![transformer-architecture](../../figures/transformer-architecture.png)

Consider a machine translation problem where English is the source language and German is the target language. The block on the left-hand side is the encoder, whose job is to take the source sentence and process it to produce a set of vector encodings that capture the meaning of the words within the source sentence, in such a way that will make it easier to translate. On the right-hand side is the decoder part of the model, which receives the encodings from the encoder that gives it the information it needs to produce the translation into the target language.

Receiving the input, our first task is to tokenise the inputs and convert the sentences into sequences of integer tokens (embedding). A common way to do this, as in RNNs, is to fix a maximum vocabulary size, and then take the most popular words seen in the corpus to populate the vocabulary. For each word in the vocabulary, we assign an integer index token. However, a Transformer uses some thing called byte-pair encoding. It can efficiently handle cases where words are concatenated together, which often happens in natural language. BPE can handle this because it recognises and tokenises the subwords that it sees. 

In the Transformer, the same tokenisation is applied to both the source and the target languages. In other words, we use a shared vocabulary for boh source and target languages. The encoding scheme will have a maximum vocabulary size that is a hyperparameter we need to set.

In any case, the tokenisation stage has the effect of encoding our long sequences of text inputs into a compressed sequence of symbols. It is these symbols that the Transformer actually uses to produce this sequence of integer tokens. Once we have this sequence of token indices, we can then use them in a lookup table of word embeddings.

![transformer-input-embeddings](../../figures/transformer-input_embeddings.png)

Each of the word embeddings would be a vector, which we will assume is of size $d_{model}$. This is a fixed hidden vector size that is used throughout the Transformer.

This input embeddings components of the Transformer will output a variable-length sequence of embedding vectors in $\mathbb R^{d_model}$, which you can think of as a hidden representation. We will use the notation $\boldsymbol h_t$ for the hidden vector representation at time step $t$. This hidden layer will then get sent on to the following layers in the encoder block of the Transformer.

![transformer-output-embeddings](../../figures/transformer-output_embeddings.png)

This process is the same for the target or output sentences. Again, we use the same BPE scheme for tokenisation as we use for the input sentences, and also the same embedding lookup table to produce the sequence of embedding vectors. The target sentences could be a different length to the source sentences.

### Multi-head Attention

The multi-head attention block is a key part of the Transformer model, and is what allows the Transformer to learn to focus on relevant parts of the input data in order to make its predictions. This mechanism also makes the model more interpretable, and allows us to analyse a trained Transformer model to see which parts of the data it is attending to at each stage of inference, adn the concepts that it connects together.

![transformer-attention](../../figures/transformer-attention.png)

The attention block operates on vectors called queries, keys and values. The idea is that each step in the sequence of hidden vectors emits a query vector. This query vector is then compared to the set of key vectors, to see which ones match well. Each key vector is paired with a value vector. For those key vectors that are a good match for the query, the corresponding value vectors get upweighted and sent on through the model for further processing.

These vectors come from the hidden layers of the model, in particular the hidden vectors $\boldsymbol h_t\in\mathbb R^{d_{model}}$ for each $t$. In the original implementation, $d_{model}=512$. We denote the key, value and query vectors with $k_t$, $v_t$, and $q_t$, respectively. The simplest case is to juse use the incoming hidden vector $\boldsymbol h_t$ for each of the key, value, and query vectors, and use these values for the attention mechanism. The Transformer actually uses a further transformation to compute the query, key and value vectors, and in the general case, the query and key vectors will have size $d_k$ and the value vectors will have size $d_v$. For now, suppose that $q_t=k_t=v_t=\boldsymbol h_t$.

In any case, at time step $t$, we have a query vector $q_t$. We compare this query vector with all of the key vectors at every time step. Then we compute a matching score between the query $q_t$ and each of these key vectors. There are choices of how to compute this matching (or similarity) score, but conventionally, the Transformer computes this with a standard inner product between these pairs of vectors, to produce a similarity score $\bar \alpha_{t,i}$:

$$
\begin{align}
\bar a_{t,i}\coloneqq \langle q_t, k_t \rangle
\end{align}
$$

for $i,t=1,\cdots,n$, where $n$ is the length of the sequence. This will give us a similarity score at each time step for the query vector $q_t$, for $t=1,\cdots,n$. These scores could be any real value, and so the next step is to normalise this set of scores so they are all positive and sum to one. This is computed using a softmax function:

$$
\begin{align}
\alpha_{t,i} = \frac{\exp(\bar \alpha_{t,i} / \sqrt{d_k})}{\sum_{j=1}^n\exp(\bar \alpha_{t,j}/\sqrt{d_k})}
\end{align}
$$

The normalised scores $\alpha_{t,i}$ are the weightings that we apply in the attention mechanism, that tells us where the Transformer shoudl be looking in the sequence for relevant information. These weights are then used to compute a weighted combination of the value vectors at each time step:

$$
\begin{align}
\boldsymbol h_t\leftarrow \sum_{j=1}^n \alpha_{t,j}v_j
\end{align}
$$

This is then the ouptut of the attention block for this time step $t$. This is the updated representation for this word token in the sequence, and this is how the representation can take context of the word into account.

This attention computation is carreid out for each of the time steps in the sequence. We can write the full collection of attention computations in the following compact form:

$$
\begin{align}
\text{Attention}(Q,K,V)=\text{softmax}\left( \frac{QK^T}{\sqrt{d_k}} \right)V
\end{align}
$$

where the matrix $Q\in\mathbb R^{n\times d_k}$ is constructed by stacking the query vectors $q_t$ in the rows, for $t=1,\cdots, n$. Similarly, the rows of $K\in\mathbb R^{n\times d_k}$ contain the key vectors $k_t$ for $t=1,\cdots,n$ and $V\in\mathbb R^{n\times d_v}$ contain the value vectors $v_t$ for $t=1,\cdots,n$.

Note that in the attention computation, the value vectors can in principle be a different size to the key and query vectors, but in practice we usually take them to be the same, so that $d_k=d_v$.

The multi-head part of the attention block comes from the observation that there could be multiple parts of the sequence that are relevant, that we would like the model to be able to focus on. Instead of just a single attention block, the multi-head atetntion block uses several attention heads, so the model can focus its attention on multiple places at once. If we only use one attention head, then the model would learn to average over all the features that are relevant. But with multiple attention heads, it is able to focus with much better resolution in multiple areas.In the original paper, the authors used 8 attention heads in each multi-head attention block.

The multi-head attention block uses a collection of lienar projection matrices to compute query, key, and value vectors for each attention head:

$$
\begin{align}
\text{head}_t = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
\end{align}
$$

where $W_i^Q, W_i^K\in\mathbb R^{d_{model}\times d_k}$ and $W_i^V\in\mathbb R^{d_{model}\times d_v}$ for $i=1,\cdots,n_h$ where $n_h$ is the number of attention heads. Each of these linear projection matrices $W_i^Q$, $W_i^K$, and $W_i^V$ are learned during training.

In the original implementation, $d_v=d_k=64=d_{model}/n_h$ with the number of attention heads $n_h=8$.

As before, the matrix $Q\in\mathbb R^{n\times d_k}$ is constructed by stacking the query vectors $q_t$ in the rows, for $t=1,\cdots,n$ and similarly for $K\in\mathbb R^{n\times d_k}$ and $V\in\mathbb R^{n\times d_v}$. Therefore, each head wil loutput $n$ vectors of size $d_v$.

The final stage is then to take the outputs from each attention head, and concatenate them:

$$
\begin{align}
\text{Multihead}(Q,K,V) = \text{concat}(\text{head}_1,\cdots,\text{head}_{n_h})W^O
\end{align}
$$

The concatenation of the output of each attention head produces a matrix of size $n\times n_hd_v$, which is equal to $n\times d_{model}$ with the choice $d_v=d_{model}/n_h$.

We then take one more linear projection $W_O\in\mathbb R^{n_hd_v\times d_{model}}$ to project back to the model dimensionality $d_{model}$. If we take the dimensions as in the original paper, then this is a square matrix, but in general it need not be.

In summary, the main points are that the attention mechanism gives the model a way of updating each word representation in the sequence so that it can take the context of the full sentence input into account. We use multiple attention heads inside this block so that the model can focus its attention on different parts of the sequence at once.

#### Softmax Scaling

Recall that in 

$$
\begin{align}
\alpha_{t,i} = \frac{\exp(\bar \alpha_{t,i} / \sqrt{d_k})}{\sum_{j=1}^n\exp(\bar \alpha_{t,j}/\sqrt{d_k})}
\end{align}
$$

there is an extra scaling in the softmax, where we divide all the $\bar \alpha_{t,i}$ values by $\sqrt{d_k}$, where $d_k$ is the size of the query and key vectors.

The reason we do this is to normalise the inputs into the softmax so that we prevent gradient vanishing. By scaling the $\bar \alpha_{t,i}$ values by $1/\sqrt{d_k}$, we obtain

$$
\begin{align}
\text{Var}(\bar {\alpha}_{t,i} / \sqrt{d_k}) = 1
\end{align}
$$

resulting in a non-saturated softmax that the gradient can more easily flow through.

### Positional Encoding

The input (and output) embedding block takes a sequence of words and tokenises them into integer tokens. Each integer token corresponds to its own embedding vector, so the embedding block outputs a sequence of embedding vectors each of size $d_{model}$. We think of this as being a matrix of size $n\times d_{model}$, where $n$ is the nubmer of tokens in the input sequence.

Before this matrix of embedding vectors is sent into the encoder (or decoder) block, we add some extra information with the positional encoding stage. This is necessary due to the way the computations are carried out in the multi-head attention block. 

The order of the input vector sequence is irrelevant within this multi-head attention block. In other words, for a set of query, key and value vectors, if we permute the order of the vectors, then the output of the multi-head attention block would be unchanged. That is, the multi-head attention block is permutation invariant.

This is a problem, since the order of the words in the input sequence is important. We should not expect that if we randomly jumble up the words in a natural language sentence that we could still identify the correct meaning and translation.

This is the reason that we have to include some information about the order of the words within the sequence of vectors that are being sent in to the multi-head attention block, so that the learned projections in the attention computation can take the order of the words into account. This information is provided by the additional positional encodings.



### Layer Normalisation

The Transformer does not use batch normalisation, but instead uses layer normalisation. Normalising the activations increases the stability of the training process as well as the speed. 

First, layer normalisation does not depend on the batch size during training. Second, it uses the same computation at both training and test time. 



We compute the activation statistics

$$
\begin{gather}
\mu_i^{(k)}=\frac{1}{n_k}\sum_{j=1}^{n_k} h_{ij}^{(k)}
\\
\left( \sigma_i^{(k)} \right)^2 = \frac{1}{n_k} \sum_{j=1}^{n_k}\left( h_{ij}^{(k)} - \mu_i^{(k)} \right)^2
\end{gather}
$$

where $h_{ij}^{(k)}$ is the post-activation for the $j$-th neuron in the $k$-th layer of the network, in the $i$-th example in the mini batch. We then normalise and apply learned parameters:

$$
\begin{gather}
\hat h_j^{(k)}=\frac{h_j^{(k)} - \mu_i^{(k)}}{\sqrt{\left( \sigma_i^{(k)} \right)^2 + \epsilon}}
\\
z_j^{(k)}=\gamma_j^{(k)} \hat h_j^{(k)} + \beta_j^{(k)}
\end{gather}
$$

where $n_k$ is the number of neurons in the $k$-th hidden layer of the network. These mean and variacne statistics are computed individually for each example in the mini batch


Once we have these layer statistics, we then normalise the post activation values in layer $k$.

The resulting normalisation results in the post activations of neurons within a layer having mean of zero and standard deviation of one, instead of having a mean of zero and standard deviation of one across the mini-batch. We end up with a more controlled spread of activation values which makes the training easier.

![transformer-layer-normalisation](../../figures/transformer-layer-normalisation.png)

The layer normalisation operates on the output of the multi-head attention block, but also includes a residual connection. This means that the sequence of input hidden vectors are added to the sequence of output vectors coming out of the multi-head attention block. This addition is the residual (or skip) connection. We then layer-normalise each hidden vector in the sequence. This can be thought of as applying layer normalisation separately for each time step. The learned parameters of layer normalisation are shared across the time steps. This then gives us the final sequence of hidden vectors.

### Transformer Architecture